In [1]:
import numpy as np

KAPPA_MAG = np.arctanh(np.sqrt(0.95)) / 2000 * 2 / np.pi


def get_L(delta_k1, delta_k2):
    """線形演算子 L を定義する"""
    return 1j * np.array([0.0, delta_k1, delta_k1 + delta_k2])


def N(B, kappa):
    """非線形項 N(B) を計算する"""
    B1, B2, B3 = B
    # 複素数として扱うことを明示
    return 1j * kappa * np.array([
        np.conj(B1) * B2 + np.conj(B2) * B3,
        B1**2 + 2 * np.conj(B1) * B3,
        3 * B1 * B2
    ], dtype=complex)


def A_from_B(B, z, delta_k1, delta_k2):
    """正準変数 B から物理変数 A に変換する"""
    phase_factors = np.exp(-1j *
                           np.array([0, delta_k1 * z, (delta_k1 + delta_k2) * z]))
    return B * phase_factors


def generate_periodic_domains(z_start, z_end, period, kappa_mag):
    """周期的分極反転構造のドメインリストを生成する"""
    domains = []
    if period < 1e-12:
        return domains
    half_period = period / 2.0
    current_z = z_start
    sign_flipper = 1.0
    # z_end を超えないようにドメインを追加
    while current_z + half_period <= z_end + 1e-9:  # 数値誤差を許容
        domains.append((half_period, sign_flipper * kappa_mag))
        current_z += half_period
        sign_flipper *= -1.0
    return domains


def calculate_phi_coeffs(L, h):
    """指数積分法で用いるφ関数を計算する"""
    z = h * L
    exp_z = np.exp(z)
    mask = np.abs(z) > 1e-9  # ゼロ割を避けるためのマスク

    phi1 = np.full_like(z, 1.0, dtype=complex)  # z=0 の極限値は 1
    phi1[mask] = (exp_z[mask] - 1.0) / z[mask]

    return exp_z, phi1


def calculate_jacobian_N(B, kappa):
    """非線形項 N(B) のヤコビ行列 J_ij = ∂N_i/∂B_j を計算する"""
    B1, B2, B3 = B
    # ここでのヤコビアンは、B と B* を独立変数とみなすWirtinger微分の考え方に基づき、
    # B に関する偏微分のみを計算します。
    # これがEKahanスキームの線形化で必要となる行列です。
    J = 1j * kappa * np.array([
        # dN1/dB1, dN1/dB2, dN1/dB3
        [0.0,          np.conj(B1), np.conj(B2)],
        # dN2/dB1, dN2/dB2, dN2/dB3
        [2 * B1,       0.0,         2 * np.conj(B1)],
        # dN3/dB1, dN3/dB2, dN3/dB3
        [3 * B2,       3 * B1,      0.0]
    ], dtype=complex)
    return J


def ekahan_step(B_n, h, kappa_val, L):
    """
    EKahanスキームの1ステップを線形方程式を直接解くことで計算する。
    これが最も高速で正確な方法。
    """
    if h < 1e-12:
        return B_n

    # 1. 必要な係数を計算
    exp_hL, phi1_hL_vec = calculate_phi_coeffs(L, h)
    phi1_hL_mat = np.diag(phi1_hL_vec)  # ベクトルを対角行列に変換
    N_n = N(B_n, kappa_val)

    # 2. ヤコビ行列を現在の点 B_n で計算
    jacobian = calculate_jacobian_N(B_n, kappa_val)

    # 3. 線形方程式 M * B_next = V を構築
    # M = I - (h/2) * φ1(hL) * N'(B_n)
    M = np.eye(3, dtype=complex) - (h / 2.0) * phi1_hL_mat @ jacobian

    # V = exp(hL)B_n + h * φ1(hL) * (N(B_n) - (1/2) * N'(B_n) * B_n)
    V = exp_hL * B_n + h * phi1_hL_mat @ (N_n - 0.5 * jacobian @ B_n)

    cond_num = np.linalg.cond(M)
    if cond_num > 1.1 or cond_num < 0.9:  # 閾値は適当
        print("Warning: M is ill-conditioned!", cond_num)

    # 4. 3x3の複素線形方程式を解く
    B_next = np.linalg.solve(M, V)

    return B_next


def calculate_propagation(B0, domains, L, subdivisions=10):
    """
    ドメイン全体を伝播する計算。
    """
    z = 0.0
    B = B0.astype(complex)
    for h_domain, kappa_val in domains:
        if h_domain < 1e-12:
            continue

        h_step = h_domain / subdivisions

        for _ in range(subdivisions):
            B = ekahan_step(B, h_step, kappa_val, L)
            z += h_step

    return z, B


def get_scenarios(kappa_mag):
    """3つのシミュレーションシナリオを定義"""
    DELTA_K1_SHG = 2 * np.pi / 7.2
    DELTA_K2_SFG = 3.2071
    period_shg = 2 * np.pi / DELTA_K1_SHG
    period_sfg = 2 * np.pi / DELTA_K2_SFG
    Z_MAX_1, Z_MAX_2, Z_MAX_3, Z_SPLIT = 2000.0, 2400.0, 4400.0, 2000.0

    return [
        {
            "name": f"Case 1: QPM for SHG (Λ={period_shg:.2f} μm)",
            "domains": generate_periodic_domains(0.0, Z_MAX_1, period_shg, kappa_mag),
            "A0": np.array([1.0, 0.0, 0.0]),
            "delta_k1": DELTA_K1_SHG, "delta_k2": DELTA_K2_SFG
        },
        {
            "name": f"Case 2: QPM for SFG (Λ≈{period_sfg:.2f} μm)",
            "domains": generate_periodic_domains(0.0, Z_MAX_2, period_sfg, kappa_mag),
            "A0": np.array([np.sqrt(0.5), np.sqrt(0.5), 0.0]),
            "delta_k1": DELTA_K1_SHG, "delta_k2": DELTA_K2_SFG
        },
        {
            "name": "Case 3: Cascaded QPM for THG",
            "domains": (
                generate_periodic_domains(0.0, Z_SPLIT, period_shg, kappa_mag) +
                generate_periodic_domains(
                    Z_SPLIT, Z_MAX_3, period_sfg, kappa_mag)
            ),
            "A0": np.array([1.0, 0.0, 0.0]),
            "delta_k1": DELTA_K1_SHG, "delta_k2": DELTA_K2_SFG
        }
    ]


if __name__ == "__main__":
    scenarios = get_scenarios(KAPPA_MAG)
    # 各ドメイン内を計算する際の分割数
    SUBDIVISIONS_PER_DOMAIN = 3

    for config in scenarios:
        L = get_L(config["delta_k1"], config["delta_k2"])

        # Scipy版の伝播計算関数を呼び出す
        final_z, final_B = calculate_propagation(
            config["A0"], config["domains"], L, subdivisions=SUBDIVISIONS_PER_DOMAIN
        )

        final_A = A_from_B(
            final_B, final_z, config["delta_k1"], config["delta_k2"])
        final_I = np.abs(final_A)**2

        print(f"\n--- {config['name']} ---")
        print(f"Final z: {final_z:.2f} μm")
        print(
            f"Final Intensities (I1, I2, I3): {final_I[0]:.4f}, {final_I[1]:.4f}, {final_I[2]:.4f}")
        print(f"Total Intensity: {np.sum(final_I):.4f}")


--- Case 1: QPM for SHG (Λ=7.20 μm) ---
Final z: 1998.00 μm
Final Intensities (I1, I2, I3): 0.5297, 0.5185, 0.0000
Total Intensity: 1.0482

--- Case 2: QPM for SFG (Λ≈1.96 μm) ---
Final z: 2399.96 μm
Final Intensities (I1, I2, I3): 0.2587, 0.0124, 0.8570
Total Intensity: 1.1281

--- Case 3: Cascaded QPM for THG ---
Final z: 4397.96 μm
Final Intensities (I1, I2, I3): 0.2767, 0.0074, 0.8984
Total Intensity: 1.1825
